In [1]:
import time
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import os

In [2]:
from datetime import datetime

In [3]:
from pandas.io import sql
from sqlalchemy import create_engine
import sqlalchemy
import mysql.connector

In [4]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import re

In [5]:
path_notebook = os.getcwd()

In [6]:
#Inicio da execução do notebook
start_general = datetime.now()

## Definir funções

In [7]:
def cadastrar_fonte_dados (nome_site, site_fonte_dados, nome_fonte_dados):
    
    fonte_dados = [[nome_site, site_fonte_dados, nome_fonte_dados]]

    colunas = ['Nome_Site', 'Site_Fonte_Dados', 'Nome_Fonte_Dados']

    fonte_dados = pd.DataFrame(fonte_dados, columns = colunas)

    with engine.connect() as conn, conn.begin():
        fonte_dados.to_sql('stg_Fonte_Dados', conn, if_exists='replace', index=False)


    ## Executar procedure versionamento

    conexao = engine.connect()

    consulta_sql = "CALL Stage.Versiona_stg_Fonte_Dados_Internet();"

    with engine.begin() as conn:
        conn.execute(consulta_sql)


    ## Deletar dados tabela stage

    consulta_sql = """
        DELETE FROM Stage.stg_Fonte_Dados
        WHERE TRUE;
    """

    with engine.begin() as conn:
        conn.execute(consulta_sql)

    conexao.close()

    ### Identificar ID Fonte de dados

    conn = engine.connect()

    consulta_sql = """
        SELECT id_Fonte_Dados
        FROM Internet_db.Fonte_Dados_Internet
        WHERE Site_Fonte_Dados = '""" +  url  + """'

    """

    with engine.begin() as conn:
        result_query = conn.execute(consulta_sql).fetchall()

    return result_query[0][0]

In [8]:
def consultar_fonte_dados(nome_site, site_fonte_dados, nome_fonte_dados):
    
    conn = engine.connect()

    consulta_sql = """
        SELECT id_Fonte_Dados
        FROM Internet_db.Fonte_Dados_Internet
        WHERE Site_Fonte_Dados = '""" +  site_fonte_dados  + """'

    """

    with engine.begin() as conn:
        result_query = conn.execute(consulta_sql).fetchall()


    if len(result_query) != 0:
        return result_query[0][0]

    else:
        print('Fonte de dados não está previamente cadastrada')
        print('Fonte de dados será cadastrada agora')

        return cadastrar_fonte_dados (nome_site, site_fonte_dados, nome_fonte_dados)

## Criar conexão banco dados

In [9]:
#texto = open('/root/airflow/scripts/config/conexao_db.txt')
texto = open('conexao_db.txt')
conexao_db = texto.read()
texto.close()

In [10]:
engine = create_engine(conexao_db, encoding='utf8')

In [11]:
print(engine.table_names())

['stg_Capital_Country', 'stg_Code_Country', 'stg_Country', 'stg_Fonte_Dados', 'stg_Population_Country']


# Importar base de dados de lista de paises

# Importar base de dados de população por país site worldpopulationreview

In [12]:
session_requests = requests.session()

In [13]:
url = 'https://worldpopulationreview.com/countries'
consulta = session_requests.get(url)
data_extracao = datetime.now()

In [14]:
consulta.ok # Will tell us if the last request was ok
consulta.status_code # Will give us the status from the last request

200

### Identificar ID Fonte de dados

In [15]:
nome_site = 'worldpopulationreview'
site_fonte_dados = url
nome_fonte_dados = 'countries'

In [16]:
id_fonte_dados = consultar_fonte_dados(nome_site, site_fonte_dados, nome_fonte_dados)

In [17]:
print('id_fonte_dados:' , id_fonte_dados)

id_fonte_dados: 2


## Conversões

In [18]:
soup = BeautifulSoup(consulta.content, 'html.parser')

In [19]:
lista_pais = pd.read_html(soup.prettify())[0]

In [20]:
lista_pais.rename(columns = {
    lista_pais.columns[1] : 'Country_Name'
}, inplace = True) 

In [21]:
lista_pais = lista_pais[['Country_Name']]

In [22]:
lista_pais['id_Fonte_Dados'] = id_fonte_dados

In [23]:
lista_pais['Data_Inicio_Vigencia'] = datetime.now().strftime('%Y-%m-%d')

In [24]:
lista_pais['Data_Fim_Vigencia'] = '9999-12-31'

In [25]:
with engine.connect() as conn, conn.begin():
    lista_pais.to_sql('stg_Country', conn, if_exists='replace', index=False)

## Executar procedure versionamento

In [26]:
conexao = engine.connect()

In [27]:
consulta_sql = "CALL Stage.Versiona_stg_Country();"

In [28]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

## Deletar dados tabela stage

In [29]:
consulta_sql = """
    DELETE FROM Stage.stg_Country
    WHERE TRUE;
"""

In [30]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

In [31]:
conexao.close()

# Importar base de dados de população por país site worldpopulationreview

## Conversões

In [32]:
soup = BeautifulSoup(consulta.content, 'html.parser')

In [33]:
populacao = pd.read_html(soup.prettify())[0]

In [34]:
# identificar ano referência
ano_referencia = populacao.columns[2]
ano_referencia = re.search('[0-9]{4}', ano_referencia).group()
print('ano_referencia:', ano_referencia)

ano_referencia: 2021


In [35]:
populacao.rename(columns = {
    populacao.columns[0] : 'Rank_Population'
    ,populacao.columns[1] : 'Country'
    ,populacao.columns[2] : 'Population'
    ,populacao.columns[3] : 'Population_last_year'
    ,populacao.columns[5] : 'Land_Area'
}, inplace = True) 

In [36]:
populacao['id_Fonte_Dados'] = id_fonte_dados

In [37]:
populacao['Ano_Referencia'] = int(ano_referencia)

In [38]:
populacao['Data_Extracao'] = data_extracao

In [39]:
populacao['Net_Change'] = populacao['Population'] - populacao['Population_last_year']

In [40]:
# Selecionar colunas
populacao = populacao[[
    'id_Fonte_Dados'
    ,'Rank_Population'
    ,'Country'
    ,'Population'
    ,'Net_Change'
    ,'Land_Area'
    ,'Ano_Referencia'
    ,'Data_Extracao'
]]

In [41]:
with engine.connect() as conn, conn.begin():
    populacao.to_sql('stg_Population_Country', conn, if_exists='replace', index=False)

## Executar procedure versionamento

In [42]:
conexao = engine.connect()

In [43]:
consulta_sql = "CALL Stage.Versiona_stg_Population_Country();"

In [44]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

## Deletar dados tabela stage

In [45]:
consulta_sql = """
    DELETE FROM Stage.stg_Population_Country
    WHERE TRUE;
"""

In [46]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

In [47]:
conexao.close()

# Importar base de dados de capitais de país

In [48]:
url = 'https://worldpopulationreview.com/continents/capitals/world'
consulta = session_requests.get(url)
data_extracao = datetime.now()

In [49]:
consulta.ok # Will tell us if the last request was ok
consulta.status_code # Will give us the status from the last request

200

### Identificar ID Fonte de dados

In [50]:
nome_site = 'worldpopulationreview'
site_fonte_dados = url
nome_fonte_dados = 'capitals/world'

In [51]:
id_fonte_dados = consultar_fonte_dados(nome_site, site_fonte_dados, nome_fonte_dados)

In [52]:
print('id_fonte_dados:' , id_fonte_dados)

id_fonte_dados: 3


## Conversões

In [53]:
soup = BeautifulSoup(consulta.content, 'html.parser')

In [54]:
capitais = pd.read_html(soup.prettify().encode('utf_8'))[0]

In [55]:
capitais['id_Fonte_Dados'] = id_fonte_dados

In [56]:
# extrair ano referência código da página
capitais['Ano_Referencia'] = datetime.now().year

In [57]:
capitais['Data_Extracao'] = data_extracao

In [58]:
capitais.iloc[[76, 142]]

,Country,Capital,id_Fonte_Dados,Ano_Referencia,Data_Extracao
76,French Polynesia,Papeetē,3,2021,2021-07-10 11:47:00.668566
142,Moldova,Chișinău,3,2021,2021-07-10 11:47:00.668566


In [59]:
# definir onde será salvo informações de capitais

In [60]:
len(capitais)

248

## Deletar dados tabela anteriores da stage

In [61]:
consulta_sql = """
    DELETE FROM Stage.stg_Capital_Country
    WHERE TRUE;
"""

In [62]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

In [63]:
conexao.close()

In [64]:
capitais.iloc[[76, 142]]

,Country,Capital,id_Fonte_Dados,Ano_Referencia,Data_Extracao
76,French Polynesia,Papeetē,3,2021,2021-07-10 11:47:00.668566
142,Moldova,Chișinău,3,2021,2021-07-10 11:47:00.668566


## Inserir dados na tabela stage

In [66]:
with engine.connect() as conn, conn.begin():
    capitais.to_sql('stg_Capital_Country', conn, if_exists='append', index=False)

UnicodeEncodeError: 'charmap' codec can't encode character '\u0113' in position 6: character maps to <undefined>

## Executar procedure versionamento

In [ ]:
conexao = engine.connect()

In [ ]:
consulta_sql = "CALL Stage.Versiona_stg_Capital_Country();"

In [ ]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

## Deletar dados tabela stage

In [ ]:
consulta_sql = """
    DELETE FROM Stage.stg_Capital_Country
    WHERE TRUE;
"""

In [ ]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

In [ ]:
conexao.close()

# Importar base de dados de código de país

In [ ]:
url = 'https://worldpopulationreview.com/country-rankings/country-codes'
consulta = session_requests.get(url)
data_extracao = datetime.now()

In [ ]:
consulta.ok # Will tell us if the last request was ok
consulta.status_code # Will give us the status from the last request

### Identificar ID Fonte de dados

In [ ]:
conn = engine.connect()

In [ ]:
consulta_sql = """
    SELECT id_Fonte_Dados
    FROM Internet_db.Fonte_Dados_Internet
    WHERE Site_Fonte_Dados = '""" +  url  + """'
    
"""

In [ ]:
result_query = conn.execute(consulta_sql).fetchall()

In [ ]:
result_query[0][0]

## Conversões

In [ ]:
soup = BeautifulSoup(consulta.content, 'html.parser')

In [ ]:
codigo_pais = pd.read_html(soup.prettify().encode("utf-8"))[0]

In [ ]:
codigo_pais['id_Fonte_Dados'] = result_query[0][0]

In [ ]:
# extrair ano referência código da página
codigo_pais['Ano_Referencia'] = datetime.now().year

In [ ]:
codigo_pais['Data_Extracao'] = data_extracao

In [ ]:
codigo_pais.rename(columns = {
    codigo_pais.columns[1] : 'ISO_Alpha_2'
    ,codigo_pais.columns[2] : 'ISO_Alpha_3'
    ,codigo_pais.columns[3] : 'Digit_Code_3'
}, inplace = True) 

In [ ]:
codigo_pais.iloc[[77, 142]]

In [ ]:
codigo_pais.head(147).tail(5)

In [ ]:
# definir onde será salvo informações de capitais

In [ ]:
len(codigo_pais)

In [ ]:
with engine.connect() as conn, conn.begin():
    codigo_pais.to_sql('stg_Code_Country', conn, if_exists='append', index=False)

## Executar procedure versionamento

In [ ]:
conexao = engine.connect()

In [ ]:
consulta_sql = "CALL Stage.Versiona_stg_Capital_Country();"

In [ ]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

## Deletar dados tabela stage

In [ ]:
consulta_sql = """
    DELETE FROM Stage.stg_Capital_Country
    WHERE TRUE;
"""

In [ ]:
with engine.begin() as conn:
    conn.execute(consulta_sql)

In [ ]:
conexao.close()

In [ ]:
#Fim da execução do notebook
end_general = datetime.now()

In [ ]:
# Write the DataFrame to a BigQuery table

print("Tempo total notebook " + str(end_general - start_general))